## Exercise 1 - Time Until 5 pm tomorrow
### 1 - Corner cases when calculating time to 5PM tomorrow
 - Tomorrow is unknown (Feb 28 on an unknown year)
 - DST forward, DST backward, start or land mid-fold
 - Run program at midnight, date changes mid-execution
 - Tomorrow is calculated correctly (skipping over a 23 hour day)
 
 ### 2 - The `time_till_5pm_tomorrow` function

In [1]:
from datetime import datetime, timedelta, date, time, timezone

In [2]:
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = datetime.now()
    tomorrow = now + timedelta(days=1)
    fivepm = time(hour=17, tzinfo = now.tzinfo)
    tomorrow_fivepm = datetime.combine(tomorrow, fivepm)
    seconds_till = tomorrow_fivepm.timestamp() - now.timestamp()
    return seconds_till/3600

### 3 - Testing `time_till_5pm_tomorrow`

In [3]:
# baseline test cases, run in US Eastern Time
print("expected anwser: unknown")
print(time_till_5pm_tomorrow())

print("expected anwser: 24.0")
print(time_till_5pm_tomorrow(datetime(2024,6,18,17)))

expected anwser: unknown
25.266525310807758
expected anwser: 24.0
24.0


In [4]:
# DST test cases, run in US Eastern Time

# March 9-10, the start of DST
print("expected anwser: 23.0")
print(time_till_5pm_tomorrow(datetime(2024,3,9,17)))

# November 2-3, the end of DST
print("expected anwser: 25.0")
print(time_till_5pm_tomorrow(datetime(2024,11,2,17)))

# March 9 11:30pm, 30 minutes before DST
print("expected anwser: 16.5")
print(time_till_5pm_tomorrow(datetime(2024,3,9,23,30)))


expected anwser: 23.0
23.0
expected anwser: 25.0
25.0
expected anwser: 16.5
16.5


### 4 - Using timezone libraries
I'm using each libraries closest approximation of NYC time, and running the last four tests which should get answers: 24, 23, 25, and 16.5

In [5]:
import pytz, dateutil
from zoneinfo import ZoneInfo

In [6]:
# datetime.timezone on EST (UTC-5:00)
NYC = timezone(-timedelta(hours=5), name="America/New_York")
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
24.0
24.0
17.5


In [7]:
# pytz.timezone('America/New_York') run with tzinfo parameter
NYC = pytz.timezone('America/New_York')
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
24.0
24.0
17.5


In [8]:
# pytz.timezone('America/New_York') run with normalize
NYC = pytz.timezone('America/New_York')
print(time_till_5pm_tomorrow(NYC.normalize(datetime(2024,6, 18,17,    tzinfo = NYC))))
print(time_till_5pm_tomorrow(NYC.normalize(datetime(2024,3, 9, 17,    tzinfo = NYC))))
print(time_till_5pm_tomorrow(NYC.normalize(datetime(2024,11,2, 17,    tzinfo = NYC))))
print(time_till_5pm_tomorrow(NYC.normalize(datetime(2024,3, 9, 23, 30,tzinfo = NYC))))

23.066666666666666
24.066666666666666
23.066666666666666
17.566666666666666


Because pytz does things differently than the other packages, we get really weird results here. We would need to modify the function to use normalize and localize calls.

In [9]:
# dateutil.tz.gettz('America/New_York')
NYC = dateutil.tz.gettz('America/New_York')
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
23.0
25.0
16.5


In [10]:
# zoneinfo.ZoneInfo('America/New_York')
NYC = ZoneInfo('America/New_York')
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
23.0
25.0
16.5


### 4.5 - Conclusions
The only two that work "out of the box" are dateutil.tz and ZoneInfo. datetime.timezone doesn't support DST at all and pytz requires some major changes to the fucnction structure for it to work correctly.

### 5 - Comparing pytz and dateutil.tz
This [ganssle blog](https://blog.ganssle.io/articles/2018/03/pytz-fastest-footgun.html) summarizes it pretty well. pytz attempts to resolve to a single timezone whenever possible and so needs to be "prompted" with normalize and localize to make dynamic changes like DST. dateutil.tz on the other hand, is lazy evaluated and so can check what date it

### 6 - Trying out different libraries
I'm using the same tests with local time, zoneinfo and dateutil.tz. If there's another built in way to handle time zones in the package, I'll also test that.

### 6.1 - Arrow
Unlike the main library, it assumes that unspecified time zones are UTC. The built in time zones work on these tests, and it also interfaces correctly with the other two timezone packages. Stuff like shift and floor are handy and it looks like theres a bunch of other useful methods.

In [11]:
import arrow
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = arrow.now()
    tomorrow = now.shift(days = 1)
    tomorrow_fivepm = tomorrow.replace(hour=17).floor('hour')
    seconds_till = tomorrow_fivepm.timestamp() - now.timestamp()
    return seconds_till/3600

In [12]:
print(time_till_5pm_tomorrow())
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30)))
print("-----")
NYC = 'America/New_York'
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30,tzinfo=NYC)))
print("-----")
NYC = dateutil.tz.gettz('America/New_York')
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30,tzinfo=NYC)))
print("-----")
NYC = ZoneInfo('America/New_York')
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30,tzinfo=NYC)))

25.266505653063458
24.0
24.0
24.0
17.5
-----
24.0
23.0
25.0
16.5
-----
24.0
23.0
25.0
16.5
-----
24.0
23.0
25.0
16.5


### 6.2 - pendulum
Also defaults to UTC (but supports naives if you need them). Similarly helpful stuff to arrow. Doesn't play well with dateutil.tz for some reason. 

In [13]:
import pendulum
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = pendulum.now()
    tomorrow = now.add(days = 1)
    tomorrow_fivepm = tomorrow.replace(hour=17).start_of('hour')
    seconds_till = tomorrow_fivepm.timestamp() - now.timestamp()
    return seconds_till/3600

In [14]:
print(time_till_5pm_tomorrow())
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30)))
print("-----")
NYC = 'America/New_York'
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30,tz=NYC)))
print("-----")
NYC = dateutil.tz.gettz('America/New_York')
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30,tz=NYC)))
print("-----")
NYC = ZoneInfo('America/New_York')
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30,tz=NYC)))

25.266497981945673
24.0
24.0
24.0
17.5
-----
24.0
23.0
25.0
16.5
-----
24.0
24.0
24.0
17.5
-----
24.0
23.0
25.0
16.5


### 6.3 - whenever
I'm a huge fan of type safety (which is why I'm working with python less and less these days), so I like the philosophy. It feels like because the main focus is safety, it has less useful methods than the other two like `humanize` or `floor`. It doesn't interface with any other timezone package, but that's because it's rebuilding the system from the ground up, which is probably a good thing.

In [15]:
import whenever
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = whenever.UTCDateTime.now()
    tomorrow = now + whenever.days(1)
    tomorrow_fivepm = tomorrow.replace(hour=17, minute=0, second=0, microsecond=0)
    diff = tomorrow_fivepm - now
    return diff.in_hours()

In [16]:
print(time_till_5pm_tomorrow())
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,6 ,18,17,   tz = 'America/New_York')))
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,3 ,9 ,17,   tz = 'America/New_York')))
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,11,2 ,17,   tz = 'America/New_York')))
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,3 ,9 ,23,30,tz = 'America/New_York')))

21.266495365833332
24.0
23.0
25.0
16.5


# Exercise 2 - Date arithmetic
### 1 - Edge cases (+ my expected behavior)
 - Add 1 year to Feb 29
   - Feb 28
 - Add 1 month to Jan 31
   - Feb 28/29
 - Add 1 hour to 1:30 on the end of DST (first, second or unspecified fold)
   - default to first, first -> second, second -> 2:30
 - Add 1 day to 1:30 on the end of DST (first and second fold)
   - Just do wall clock time (so 1:30 next day in either case)
 - Remove 1 day from 1:30 on the day after the end of DST
   - First fold
 - Add 1 month, 2 days to Jan 29 (order of operations)
   - Mar 2 (larger time periods first)
 - (1 day, 15 hours) / 2
   - either invalid or 19.5 hours
 - (1 day, 15 hours) * 2
   - 3 days, 6 hours
 - difference over DST (both ways)
   - no clue what I'd expect honestly. I think I agree with [this blog](https://blog.ganssle.io/articles/2018/02/aware-datetime-arithmetic.html) that it should be absolute time, but I could see either honestly.
 - difference across time zones
   - absolute time

### 2 - Testing

In [17]:
# 1,2
# Timedelta doesn't work with years and months, so I'm seeing what happens if you just manually edit the month and year
try:
    d = date(2024,2,29)
    print(d)
    print(d.replace(year = d.year + 1))
except Exception as e:
    print(e)

try:
    d = date(2024,1,31)
    print(d)
    print(d.replace(month = d.month + 1))
except Exception as e:
    print(e)

2024-02-29
day is out of range for month
2024-01-31
day is out of range for month


In [18]:
# 3,4
NYC = dateutil.tz.gettz('America/New_York')
d0 = datetime(2024,11,3,1,30, tzinfo = NYC)
d1 = datetime(2024,11,3,1,30, fold = 0, tzinfo = NYC)
d2 = datetime(2024,11,3,1,30, fold = 1, tzinfo = NYC)

print("d0 =", d0, "=", d0.timestamp())
print("d1 =", d1, "=", d1.timestamp())
print("d2 =", d2, "=", d2.timestamp())
# Note: does default to fold = 0

print(d1 + timedelta(hours = 1))
print(d1 + timedelta(days = 1))
# unexpected behavior, skips over the extra daylight savings hour

print(d2 + timedelta(hours = 1))
print(d2 + timedelta(days = 1))

d0 = 2024-11-03 01:30:00-04:00 = 1730611800.0
d1 = 2024-11-03 01:30:00-04:00 = 1730611800.0
d2 = 2024-11-03 01:30:00-05:00 = 1730615400.0
2024-11-03 02:30:00-05:00
2024-11-04 01:30:00-05:00
2024-11-03 02:30:00-05:00
2024-11-04 01:30:00-05:00


In [19]:
# 5
NYC = dateutil.tz.gettz('America/New_York')
d = datetime(2024,11,4,1,30, tzinfo = NYC)
print(d1 + timedelta(days = -1))

2024-11-02 01:30:00-04:00


In [20]:
# 6
# invalid because the default datetime library doesn't support adding months

In [23]:
# 7, 8
td = timedelta(days=1, hours=15)
print(td)
print(td*2)
print(td/2)
print(td//2)

1 day, 15:00:00
3 days, 6:00:00
19:30:00
19:30:00


In [25]:
# 9
NYC = dateutil.tz.gettz('America/New_York')
d1 = datetime(2024,11,2,12, tzinfo = NYC)
d2 = datetime(2024,11,3,12, tzinfo = NYC)
print(d2-d1)

NYC = dateutil.tz.gettz('America/New_York')
d1 = datetime(2024,3,9,12, tzinfo = NYC)
d2 = datetime(2024,3,10,12, tzinfo = NYC)
print(d2-d1)

1 day, 0:00:00
1 day, 0:00:00


In [26]:
#10
d1 = datetime(2024,1,1, tzinfo = dateutil.tz.gettz('America/Los_Angeles'))
d2 = datetime(2024,1,1, tzinfo = dateutil.tz.gettz('America/New_York'))
print(d1-d2)

3:00:00


### 2.1 - Package arithmetic
Since the different datetime packages handle arithmetic a little bit differently, I'm going to run an abbreviated test on this.

In [ ]:
# Arrow
# TODO

In [44]:
# Pendulum
import pendulum
NYC = 'America/New_York'

print("1:", pendulum.datetime(2024, 2, 29).add(years=1))
print("2:", pendulum.datetime(2023, 1, 31).add(months=1)) # 2024 is a leap year oops
# Note: defaults to second fold
print("3:", pendulum.datetime(2024,11,3,1,30, fold=0, tz = NYC).timestamp()) 
print("3:", pendulum.datetime(2024,11,3,1,30, fold=0, tz = NYC).add(hours=1))
print("3:", pendulum.datetime(2024,11,3,1,30, fold=1, tz = NYC).add(hours=1))
print("4:", pendulum.datetime(2024,11,3,1,30, fold=0, tz = NYC).add(days=1))
print("4:", pendulum.datetime(2024,11,3,1,30, fold=1, tz = NYC).add(days=1))
print("5:", pendulum.datetime(2024,11,4,1,30, tz = NYC).add(days=-1)) # lands in second fold
print("6:", pendulum.datetime(2023, 1, 29).add(months=1, days=2)) # expected behavior
# pendulum does not have timedeltas and so 7 and 8 don't apply
print("9:", (pendulum.datetime(2024,11,3,12, tz = NYC) - pendulum.datetime(2024,11,2,12, tz = NYC)).in_hours())
print("9:", (pendulum.datetime(2024,3,10,12, tz = NYC) - pendulum.datetime(2024,3,9,12, tz = NYC)).in_hours())
print("10:", (pendulum.datetime(2024,1,1, tz = 'America/Los_Angeles') - pendulum.datetime(2024,1,1, tz = 'America/New_York')).in_hours())

1: 2025-02-28 00:00:00+00:00
2: 2023-02-28 00:00:00+00:00
3: 1730611800.0
3: 2024-11-03 01:30:00-05:00
3: 2024-11-03 02:30:00-05:00
4: 2024-11-04 01:30:00-05:00
4: 2024-11-04 01:30:00-05:00
5: 2024-11-03 01:30:00-05:00
6: 2023-03-02 00:00:00+00:00
9: 25
9: 23
9: 3


In [ ]:
# Whenever
# TODO

### 3 - Equality